In [1]:
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,mean_absolute_error
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import spiro.data_preprocessing

random.seed(0)
np.random.seed(0)

In [2]:
data = pd.read_csv("../data/feature_30_joint_51.csv")
data= data[~data.iloc[:,0].isin([21,48])].reset_index(drop=True)
data.drop("Unnamed: 0",axis=1,inplace=True)

In [3]:
labels = pd.read_csv("../data/labels.csv")

In [6]:
final_data= pd.DataFrame(data)

In [5]:
demographic_data = pd.read_csv("../data/GroundTruth_Dataset.csv")

In [7]:
PID= ['P0', 'P2', 'P3', 'P5', 'P6', 'P9', 'P10', 'P12', 'P13', 'P14', 'P15', 'P17', 'P18', 'P19', 'P20', 'P21', 'P23', 'P26', 'P27', 'P28', 'P29', 'P31', 'P32', 'P33', 'P36','P38', 'P39', 'P40', 'P41', 'P42', 'P43', 'P44', 'P45', 'P46','P50', 'P51', 'P52', 'P53', 'P55', 'P57', 'P58', 'P60', 'P61','P62', 'P63', 'P65', 'P66', 'P69', 'P70', 'P71','P73']

In [8]:
demographic_data= demographic_data[demographic_data.iloc[:,1].isin(PID)].reset_index(drop=True)
demographic_data.shape

(51, 17)

In [9]:
final_data['Sex'] = demographic_data['Sex'].map({'M': 0, 'F': 1})

In [10]:
final_data= pd.concat((final_data,pd.DataFrame(demographic_data.iloc[:,7:10])),axis=1,ignore_index=True)

In [11]:
X1 = final_data.iloc[:,:30].astype(float)
X1 = MinMaxScaler().fit_transform(np.array(X1))
X1 = pd.DataFrame(X1)
y = labels.iloc[:,-1].astype(float)


X2 = final_data.astype(float)
X2 = MinMaxScaler().fit_transform(np.array(X2))
X2 = pd.DataFrame(X2)

#For FEV1 
# y= labels.iloc[:,-2].astype(float)

In [12]:
X1.shape,X2.shape,y.shape

((51, 30), (51, 34), (51,))

In [13]:
RF = RandomForestRegressor(random_state=42)
# X_train, X_test, X_pool, Y_train, Y_test, Y_pool = data_preprocessing.train_pool_split(X[:,:20], Y, train_idx, test_idx)
# Total number of the splits
tot = len(X1)
# Instantiating Leave_One_Out split function.
loo = LeaveOneOut()

prog        = 0
y_GT        = []
y_PT        = []
abserror    = []

print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=")
# print("Random Forest Regressor for FVC with {0} features".format(i))

# For every split obtained by Leave_One_Out split function.
for i,(train_index, test_index) in enumerate(loo.split(X1)):
    
    # To show some sort of progress.
    prog = prog + 1
    print("Progress : {0}/{1}".format(prog,tot),end = '\r')

    # Test-train split for the fold.
    X_Train, X_Test = X1.iloc[train_index],X1.iloc[test_index]
    y_Train, y_Test = y.iloc[train_index],y.iloc[test_index]
    
    # Training new Random Forest ensemble.
    RF.fit(X_Train, y_Train)
    pred = RF.predict(X_Test)  

    # print("Ground Truth : {0}".format(np.array(y_Test)[0]))
    # Storing the values of Ground truth and Predicted value for future use.
    # print(np.array(y_Test)[0])
    y_GT.append(np.array(y_Test)[0])
    y_PT.append(pred[0]) 


print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")
print("Random forest metrics : ")
print("\tMean Absolute Percentage Error : " , 100 * mean_absolute_percentage_error(y_GT,y_PT))
print("\tMean Aabsolute Error : " , mean_absolute_error(y_GT,y_PT))
print("\tMean Square Error : " , mean_squared_error(y_GT,y_PT,squared=False))
print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Random forest metrics : 
	Mean Absolute Percentage Error :  4.580250399527127
	Mean Aabsolute Error :  0.15071764705882332
	Mean Square Error :  0.1915421215130638
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [14]:
RF = RandomForestRegressor(random_state=42)
# RF = GradientBoostingRegressor(random_state=42)
# RF = DecisionTreeRegressor(random_state=0)
# X_train, X_test, X_pool, Y_train, Y_test, Y_pool = data_preprocessing.train_pool_split(X[:,:20], Y, train_idx, test_idx)
# Total number of the splits
tot = len(X2)
# Instantiating Leave_One_Out split function.
loo = LeaveOneOut()

prog        = 0
y_GT_2        = []
y_PT_2        = []
abserror_2    = []

print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=")
# print("Random Forest Regressor for FVC with {0} features".format(i))

# For every split obtained by Leave_One_Out split function.
for i,(train_index, test_index) in enumerate(loo.split(X2)):
    
    # To show some sort of progress.
    prog = prog + 1
    print("Progress : {0}/{1}".format(prog,tot),end = '\r')

    # Test-train split for the fold.
    X_Train, X_Test = X2.iloc[train_index],X2.iloc[test_index]
    y_Train, y_Test = y.iloc[train_index],y.iloc[test_index]
    
    # Training new Random Forest ensemble.
    RF.fit(X_Train, y_Train)
    pred = RF.predict(X_Test)  

    # print("Ground Truth : {0}".format(np.array(y_Test)[0]))
    # Storing the values of Ground truth and Predicted value for future use.
    # print(np.array(y_Test)[0])
    y_GT_2.append(np.array(y_Test)[0])
    y_PT_2.append(pred[0]) 


print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")
print("Random forest metrics : ")
print("\tMean Absolute Percentage Error : " , 100 * mean_absolute_percentage_error(y_GT_2,y_PT_2))
print("\tMean Aabsolute Error : " , mean_absolute_error(y_GT_2,y_PT_2))
print("\tMean Square Error : " , mean_squared_error(y_GT_2,y_PT_2,squared=False))
print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=")

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Random forest metrics : 
	Mean Absolute Percentage Error :  4.029816167406599
	Mean Aabsolute Error :  0.132513725490196
	Mean Square Error :  0.17307048433808553
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [15]:
mape_error = []
for i in range(len(y_GT)):
    mape_error.append(100*(abs((y_GT[i]-y_PT[i])/y_GT[i])))

In [16]:
mape_error_2 = []
for i in range(len(y_GT_2)):
    mape_error_2.append(100*(abs((y_GT_2[i]-y_PT_2[i])/y_GT_2[i])))

In [17]:
idx = [0,7,8,9,17,18,30,31,34,35,36,37,38,39,40,45] # Female indices
# idx= [5,6,7,11,12,13,14,20,23] #Unhealthy Participants
print(len(idx))
s=0
p=0
q=0
m=0
for i in idx:
    # print("Patient {0} : {1}".format(i,mape_error[i]))
    # print(y_GT[i],y_1PT[i])
    p+= y_GT[i]
    q+= y_PT[i]
    m+=1
    s+=mape_error[i]
print("Average : {0}".format(s/len(idx)))
# print(q/m)
# print(p/m)


16
Average : 4.882766281202224


In [18]:
s=0
m=0
p=0
q=0
for i in range(len(mape_error)):
    if i not in idx:
        m+= 1
        p+= y_GT[i]
        q+= y_PT[i]
        s+=mape_error[i]
print(s/m)
print(m)


4.441957425047084
35


# With demographics

## female

In [19]:
idx = [0,7,8,9,17,18,30,31,34,35,36,37,38,39,40,45] # Female indices
# idx= [5,6,7,11,12,13,14,20,23] #Unhealthy Participants
print(len(idx))
s=0
p=0
q=0
m=0
for i in idx:
    # print("Patient {0} : {1}".format(i,mape_error[i]))
    # print(y_GT[i],y_1PT[i])
    p+= y_GT_2[i]
    q+= y_PT_2[i]
    m+=1
    s+=mape_error_2[i]
print("Average : {0}".format(s/len(idx)))
# print(q/m)
# print(p/m)


16
Average : 3.49188731719667


## Male

In [20]:
s=0
m=0
p=0
q=0
for i in range(len(mape_error_2)):
    if i not in idx:
        # print("Patient {0} : {1}".format(i,mape_error[i]))
        m+= 1
        # print(y_GT[i],y_PT[i])
        p+= y_GT_2[i]
        q+= y_PT_2[i]
        s+=mape_error_2[i]
# print(s)
# print("Average : {0}".format(s/(len(mape_error)-len(idx))))
print(s/m)
print(m)


4.275726498931139
35
